# Importing Libraries

In [1]:
# System Libraries
import os
import sys

# Database Connector
import pymysql

# Data Manipulator
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Accessory Libraries
import time

# Import custom Library
import CompData as cd

# Loading Data

### Setting up for execution

In [2]:
# Returns the keys and columns for all tables
Columns, Keys = cd.get_table_keys_columns(cd.get_tables())

# Grab columns for Training Data
Application_Column_Tapper_Train = cd.get_core_columns(Columns, 'Application_Train')

# Grab columns for Testing Data
Application_Column_Tapper_Test = cd.get_core_columns(Columns, 'Application_Test')

### Executes query to grab final data set

**To join the tables, we first need to figure out how we are going to make all the rows unique in the external table prior to the join. As such, we can go ahead and table this to the side first and proceed with the single data set before proceeding. When we decide to join the tables however, there are some complications that we want to consider. One is the Credit Activity Status, and the other is the conversion of the currencies which will allow us to normalize the debt which will grant us to aggregate the information.**

In [3]:
# Grabbing Training Data
train = cd.get_data('SELECT * FROM Application_Train')

# Grabbing Testing Data
test = cd.get_data('SELECT * FROM Application_Test')

This query took 84.49 seconds to run.
This query took 13.22 seconds to run.


# Data Processing

**First, we will have to determine the data type and we will figure out how to deal with it here. The method we are going to use here for the continuous variables are as follows:**

1) We take a look at where the data is complete

2) Try two methodologies:

    a) OLS regression

    b) Sampling + Data oscilation and rule-based assignment of values

**This should help us maintain the distribution shape and allow us to process the data better later on in the machine learning phase. For discrete values, we can try the following:**

1) To send the missing values far off into the corner

2) Apply Causal Inference to determine if we can find a 'What If' criteria of assignment

3) Sample and use rule-based assignment

**This should help us to tackle the impurities in both the testing and training data sets and will provide us with a good foot hold. Now that we have the potential solutions, we can proceed by first taking a partition of the data set using the table of missing values for the training data. We store the missing value table now for manipulation.**

### Studying Extent of Impurities

In [4]:
# Gets table of Missing Values for training set
print('''Training Data Metrics:
----------------------''')
missing_train = cd.get_missing(train)
display(missing_train)

# Retreives the columns with missing values for training set
Floats, Objects, Integers = cd.get_missing_partition(missing_train)

# Gets table of Missing Values for testing set
print('''\n\nTesting Data Metrics:
---------------------''')
missing_test = cd.get_missing(test)
display(missing_test)

# Retreives the columns with missing values for testing set
Floats, Objects, Integers = cd.get_missing_partition(missing_test)

Training Data Metrics:
----------------------


,COMMONAREA_MEDI,COMMONAREA_AVG,COMMONAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_MODE,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,FLOORSMIN_AVG,FLOORSMIN_MODE,FLOORSMIN_MEDI,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,YEARS_BUILD_AVG,OWN_CAR_AGE,LANDAREA_MEDI,LANDAREA_MODE,LANDAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_AVG,BASEMENTAREA_MODE,EXT_SOURCE_1,NONLIVINGAREA_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,ELEVATORS_MEDI,ELEVATORS_AVG,ELEVATORS_MODE,WALLSMATERIAL_MODE,APARTMENTS_MEDI,APARTMENTS_AVG,APARTMENTS_MODE,ENTRANCES_MEDI,ENTRANCES_AVG,ENTRANCES_MODE,LIVINGAREA_AVG,LIVINGAREA_MODE,LIVINGAREA_MEDI,HOUSETYPE_MODE,FLOORSMAX_MODE,FLOORSMAX_MEDI,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_AVG,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,FLAG_DOCUMENT_8,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_9,FLAG_DOCUMENT_21,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_OWN_REALTY,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_12,AMT_CREDIT,AMT_INCOME_TOTAL,FLAG_PHONE,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,TARGET,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,FLAG_EMAIL,FLAG_CONT_MOBILE,ORGANIZATION_TYPE,FLAG_WORK_PHONE,FLAG_EMP_PHONE,FLAG_MOBIL,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,SK_ID_CURR
Missing Count,214865,214865,214865,213514,213514,213514,210295,210199,210199,210199,208642,208642,208642,204488,204488,204488,202929,182590,182590,182590,179943,179943,179943,173378,169682,169682,169682,163891,163891,163891,156341,156061,156061,156061,154828,154828,154828,154350,154350,154350,154297,153020,153020,153020,150007,150007,150007,148431,145755,96391,60965,41519,41519,41519,41519,41519,41519,1292,1021,1021,1021,1021,660,278,12,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Percent,69.87%,69.87%,69.87%,69.43%,69.43%,69.43%,68.39%,68.35%,68.35%,68.35%,67.85%,67.85%,67.85%,66.5%,66.5%,66.5%,65.99%,59.38%,59.38%,59.38%,58.52%,58.52%,58.52%,56.38%,55.18%,55.18%,55.18%,53.3%,53.3%,53.3%,50.84%,50.75%,50.75%,50.75%,50.35%,50.35%,50.35%,50.19%,50.19%,50.19%,50.18%,49.76%,49.76%,49.76%,48.78%,48.78%,48.78%,48.27%,47.4%,31.35%,19.83%,13.5%,13.5%,13.5%,13.5%,13.5%,13.5%,0.42%,0.33%,0.33%,0.33%,0.33%,0.21%,0.09%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Data Types,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,


There are 61 missing float, 6 missing object, and 0 missing integer columns.


Testing Data Metrics:
---------------------


,COMMONAREA_AVG,COMMONAREA_MODE,COMMONAREA_MEDI,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MODE,LIVINGAPARTMENTS_MEDI,FLOORSMIN_MEDI,FLOORSMIN_AVG,FLOORSMIN_MODE,OWN_CAR_AGE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,LANDAREA_MEDI,LANDAREA_AVG,LANDAREA_MODE,BASEMENTAREA_MEDI,BASEMENTAREA_AVG,BASEMENTAREA_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MODE,NONLIVINGAREA_MEDI,ELEVATORS_MODE,ELEVATORS_MEDI,ELEVATORS_AVG,WALLSMATERIAL_MODE,APARTMENTS_MODE,APARTMENTS_MEDI,APARTMENTS_AVG,HOUSETYPE_MODE,ENTRANCES_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,LIVINGAREA_MEDI,LIVINGAREA_MODE,LIVINGAREA_AVG,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,TOTALAREA_MODE,EMERGENCYSTATE_MODE,EXT_SOURCE_1,OCCUPATION_TYPE,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_ANNUITY,EXT_SOURCE_2,FLAG_DOCUMENT_21,FLAG_DOCUMENT_20,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,AMT_INCOME_TOTAL,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_12,AMT_CREDIT,ORGANIZATION_TYPE,AMT_GOODS_PRICE,LIVE_CITY_NOT_WORK_CITY,NAME_CONTRACT_TYPE,REG_CITY_NOT_WORK_CITY,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,CNT_FAM_MEMBERS,FLAG_EMAIL,FLAG_PHONE,FLAG_CONT_MOBILE,FLAG_WORK_PHONE,FLAG_EMP_PHONE,FLAG_MOBIL,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,SK_ID_CURR
Missing Count,33495,33495,33495,33347,33347,33347,32797,32780,32780,32780,32466,32466,32466,32312,31818,31818,31818,28254,28254,28254,27641,27641,27641,26084,26084,26084,25189,25189,25189,23893,23887,23887,23887,23619,23579,23579,23579,23552,23552,23552,23321,23321,23321,22856,22856,22856,22624,22209,20532,15605,8668,6049,6049,6049,6049,6049,6049,911,29,29,29,29,24,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Percent,68.72%,68.72%,68.72%,68.41%,68.41%,68.41%,67.28%,67.25%,67.25%,67.25%,66.61%,66.61%,66.61%,66.29%,65.28%,65.28%,65.28%,57.96%,57.96%,57.96%,56.71%,56.71%,56.71%,53.51%,53.51%,53.51%,51.68%,51.68%,51.68%,49.02%,49.01%,49.01%,49.01%,48.46%,48.37%,48.37%,48.37%,48.32%,48.32%,48.32%,47.84%,47.84%,47.84%,46.89%,46.89%,46.89%,46.41%,45.56%,42.12%,32.01%,17.78%,12.41%,12.41%,12.41%,12.41%,12.41%,12.41%,1.87%,0.06%,0.06%,0.06%,0.06%,0.05%,0.02%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Data Types,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,int64,int64,object,object,object,int64,float64,int64,int64,i


There are 60 missing float, 4 missing object, and 0 missing integer columns.


### Processing Step 1: Cleaning Values

In [5]:
# Drop all missing values for processing
train_temp_clean = train.dropna()